In [5]:
import torch
import numpy as np
from models import LIMUBertModel4Pretrain, ClassifierGRU
from config import PretrainModelConfig, ClassifierModelConfig, load_model_config, load_dataset_stats

# Asumimos que estamos en un notebook de Colab/Jupyter

# 1. Configuración y carga de modelos
def load_models(bert_path, classifier_path, dataset, version):
    # Cargar configuraciones
    bert_cfg = load_model_config('pretrain', 'base', 'v1', path_bert='config/limu_bert.json')
    classifier_cfg = load_model_config('classifier', 'gru', 'v1', path_classifier='config/classifier.json')
    
    # Imprimir la configuración del clasificador para verificar
    print("Classifier config:", classifier_cfg)
    
    # Inicializar modelos
    bert_model = LIMUBertModel4Pretrain(bert_cfg, output_embed=True)
    # Asumiendo que el modelo guardado tiene una capa lineal final de 6 a 10
    output_dim = 6  # o el valor correcto basado en tu modelo guardado
    classifier_model = ClassifierGRU(classifier_cfg, input=bert_cfg.hidden, output=output_dim)
    #classifier_model = ClassifierGRU(classifier_cfg, input=bert_cfg.hidden, output=classifier_cfg.linear_io[-1][1])
    
    # Imprimir la estructura del modelo para verificar
    print(classifier_model)
    
    # Cargar pesos pre-entrenados
    bert_model.load_state_dict(torch.load(bert_path, map_location='cpu'))
    
    # Cargar el estado del clasificador
    classifier_state = torch.load(classifier_path, map_location='cpu')
    
    # Imprimir las claves y formas del estado del clasificador
    for key, value in classifier_state.items():
        print(f"{key}: {value.shape}")
    
    # Intenta cargar el estado del clasificador
    classifier_model.load_state_dict(classifier_state, strict=False)
    
    return bert_model, classifier_model

# 2. Preprocesamiento de datos
def preprocess_data(raw_data, window_size=120, stride=60):
    windows = []
    for i in range(0, len(raw_data) - window_size + 1, stride):
        windows.append(raw_data[i:i+window_size])
    return np.array(windows)

# 3. Inferencia
def inference(bert_model, classifier_model, input_data):
    bert_model.eval()
    classifier_model.eval()
    
    with torch.no_grad():
        embeddings = bert_model(torch.FloatTensor(input_data))
        outputs = classifier_model(embeddings)
        
    return torch.softmax(outputs, dim=1).numpy()

# 4. Función principal
def classify_imu_data(raw_data, seconds, sample_rate, bert_path, classifier_path, dataset, version):
    # Cargar modelos
    bert_model, classifier_model = load_models(bert_path, classifier_path, dataset, version)
    
    # Cargar configuración del dataset
    dataset_config = load_dataset_stats(dataset, version)
    
    # Preprocesar datos
    n_samples = seconds * sample_rate
    data = raw_data[:n_samples]  # Tomar solo los primeros N segundos
    preprocessed_data = preprocess_data(data, window_size=dataset_config.seq_len, stride=dataset_config.seq_len//2)
    
    # Realizar inferencia
    predictions = inference(bert_model, classifier_model, preprocessed_data)
    
    # Agregar predicciones
    final_prediction = np.mean(predictions, axis=0)
    
    return final_prediction



In [6]:
# Ejemplo de uso
BERT_PATH = 'saved/pretrain_base_motion_20_120/motion.pt'
CLASSIFIER_PATH = 'saved/classifier_base_gru_motion_20_120/motion.pt'
DATASET = 'hhar'  # o el nombre de tu dataset
VERSION = '20_120'  # o la versión que estés usando

# Simular datos de IMU (reemplaza esto con tus datos reales)
raw_imu_data = np.random.randn(6000, 6)  # 1 minuto de datos a 100Hz

# Clasificar 10 segundos de datos
result = classify_imu_data(raw_imu_data, seconds=10, sample_rate=100, 
                            bert_path=BERT_PATH, classifier_path=CLASSIFIER_PATH,
                            dataset=DATASET, version=VERSION)

print("Predicción final:", result)

Classifier config: ClassifierModelConfig(seq_len=20, input=6, num_rnn=2, num_layers=[2, 1], rnn_io=[[6, 20], [20, 10]], num_cnn=0, conv_io=[], pool=[], flat_num=0, num_attn=0, num_head=0, atten_hidden=0, num_linear=1, linear_io=[[10, 3]], activ=False, dropout=False)
ClassifierGRU(
  (gru0): GRU(72, 20, num_layers=2, batch_first=True)
  (gru1): GRU(20, 10, batch_first=True)
  (lin0): Linear(in_features=10, out_features=6, bias=True)
)
gru0.weight_ih_l0: torch.Size([60, 72])
gru0.weight_hh_l0: torch.Size([60, 20])
gru0.bias_ih_l0: torch.Size([60])
gru0.bias_hh_l0: torch.Size([60])
gru0.weight_ih_l1: torch.Size([60, 20])
gru0.weight_hh_l1: torch.Size([60, 20])
gru0.bias_ih_l1: torch.Size([60])
gru0.bias_hh_l1: torch.Size([60])
gru1.weight_ih_l0: torch.Size([30, 20])
gru1.weight_hh_l0: torch.Size([30, 10])
gru1.bias_ih_l0: torch.Size([30])
gru1.bias_hh_l0: torch.Size([30])
lin0.weight: torch.Size([6, 10])
lin0.bias: torch.Size([6])
Predicción final: [0.49981365 0.16578202 0.01194162 0.0640